In [ ]:
import panel as pn
pn.extension()
pn.config.sizing_mode="stretch_width"

The ``Location`` widget allows getting and setting the parts of the url (``href``) of the browser.

The ``Location`` widgets enables implementing

- Navigation between pages via ``pathname``
- Sharing (parts of) the page state in the url as ``search`` parameters for bookmarking and sharing.
- Navigating to subsections of the page via the ``hash_`` parameter.

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

##### Core

* **``pathname``** (string): pathname part of the url, e.g. '/user_guide/Interact.html'.
* **``search``** (string): search part of the url e.g. '?color=blue'.
* **``hash_``** (string): hash part of the url e.g. '#interact'.
* **``reload``** (bool): Whether or not to reload the page when the url is updated.
    - For independent apps this should be set to True. 
    - For integrated or single page apps this should be set to False.

##### Readonly

* **``href``** (string): The full url, e.g. 'https://panel.holoviz.org/user_guide/Interact.html:80?color=blue#interact'.
* **``protocol``** (string): protocol part of the url, e.g. 'http:' or 'https:'
* **``port``** (string): port number, e.g. '80'

___

The Location widget does not display anything by it self. But it needs to be added to a layout in order to work.

In [ ]:
location = pn.widgets.Location(reload=False)
parameters = [
    "href",
    "hostname",
    "pathname",
    "protocol",
    "port",
    "search",
    "hash_",
    "reload",
]
pn.Column(location, pn.Param(location, parameters=parameters))

## Using the Location Widget in an App with Multiple Pages

In [ ]:
import param
from urllib import parse

class Page(param.Parameterized):
    title = param.String("new page")
    value = param.Number(default=0.0, bounds=None, step=1)

page1 = Page(title="#### Page 1", name="page1")
page2 = Page(title="#### Page 2", name="page2")
pages = [page1, page2]

location = pn.widgets.Location(reload=False)

class App(param.Parameterized):
    page = param.ObjectSelector(page1, objects=pages)
    location = param.Parameter(location)
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.view = pn.Column(
            self.location, 
            "#### Menu", 
            pn.Param(
                self, 
                parameters=["page"], 
                expand_button=False, 
                show_name=False), 
                self._selected_page
        )
        
    @param.depends("page")
    def _selected_page(self):
        return pn.Column(self.page.title, self.page.param.value)
    
    @param.depends("page", watch=True)
    def _set_pathname(self):
        location.pathname = "/" + self.page.name
        
    @param.depends("page", "page.value", watch=True)
    def _set_search(self):
        location.search = "?value=" + self.page.value
        
    @param.depends("location", "location.search", watch=True)
    def _set_page_value(self):
        if search:
            search=search[1:]
            search_dict=dict(parse.parse_qsl(search))
        else:
            search_dict={}

        if "value" in search_dict:
            self.page.value = int(search_dict["value"])
            
app = App()       
app.view